In [ ]:
from Utils.help_func import *
import warnings
import heapq

warnings.filterwarnings("ignore")

In [ ]:
data = sc.read_h5ad('./Processed Data/' + data_name + '.h5ad'

In [ ]:
peak_embedding = pd.read_csv('./Results/SOX10_SCARP_Peaks_feature.csv', index_col=0)

####  filter peaks co-accessible with the promoter of SOX10

In [ ]:
SOX10_corr_promoter = [np.corrcoef(peak_mat.iloc[np.where(data.var.index == 'chr22:38380499-38380726')[0][0], :],
                                   peak_mat.iloc[i, :])[0, 1] for i in range(peak_mat.shape[0])]
SOX10_promoter_index = heapq.nlargest(10, range(len(SOX10_corr_promoter)), SOX10_corr_promoter.__getitem__)
diff_access_sites_promoter = peak_mat.iloc[SOX10_promoter_index, :].index

#### filter peaks co-accessible with the 3'UTR of SOX10

In [ ]:
SOX10_corr_3UTR = [np.corrcoef(peak_mat.iloc[np.where(data.var.index == 'chr22:38364975-38365257')[0][0], :],
                               peak_mat.iloc[i, :])[0, 1] for i in range(peak_mat.shape[0])]
SOX10_3UTR_index = heapq.nlargest(10, range(len(SOX10_corr_3UTR)), SOX10_corr_3UTR.__getitem__)
diff_access_sites_3UTR = peak_mat.iloc[SOX10_3UTR_index, :].index

#### filter peaks co-accessible with SOX10

In [ ]:
diff_access_sites = np.hstack([np.array(diff_access_sites_promoter),
                               np.array(diff_access_sites_3UTR)])

#### filter peaks co-accessible with SOX10

In [ ]:
data.X = (data.X > 0) * 1
data_df = pd.DataFrame(np.array(data.X.todense()),
                       index=data.obs_names,
                       columns=data.var_names)
data1_df = data_df[peak_mat.index]

In [ ]:
data1 = sc.AnnData(data1_df)
data1.var_names_make_unique()
data1.obs['celltype'] = data.obs['celltype']

In [ ]:
fig1, ax = plt.subplots(figsize=(10, 5))
plt.subplots_adjust(left=0.13, right=0.95, top=1, bottom=0.4)
dp = sc.pl.dotplot(data1, {'Loci co-accessible with SOX10(promoter)':
                               diff_access_sites[:10],
                           'Loci co-accessible with SOX10(3\'UTR)':
                               diff_access_sites[10:]},
                   groupby='celltype', var_group_rotation=0,
                   colorbar_title='Mean accessibility\nin group',
                   title='Co-accessible loci of SOX10 computed by SCARP peak feature',
                   ax=ax)

In [ ]:
annotation = pd.read_table('../SCARP_R_downstream_analysis/SOX10 data/annotated_genes_Promoter_1000bp.csv',
                           sep=',', index_col=0)
annotation['peak_name'] = [annotation['seqnames'].iloc[i] + ':' +
                           str(annotation['start'].iloc[i] - 1) +
                           '-' + str(annotation['end'].iloc[i])
                           for i in range(annotation.shape[0])]
annotation.index = annotation['peak_name']

In [ ]:
# =============================================================
#               前100个
# =========================================================
SOX10_promoter_index = heapq.nlargest(100, range(len(SOX10_corr_promoter)), SOX10_corr_promoter.__getitem__)
diff_access_sites_promoter = peak_mat.iloc[SOX10_promoter_index, :].index

SOX10_3UTR_index = heapq.nlargest(100, range(len(SOX10_corr_3UTR)), SOX10_corr_3UTR.__getitem__)
diff_access_sites_3UTR = peak_mat.iloc[SOX10_3UTR_index, :].index

# merge
diff_access_sites = np.hstack([np.array(diff_access_sites_promoter),
                               np.array(diff_access_sites_3UTR)])

diff_access_sites_df = pd.DataFrame(np.hstack([np.array(['pos' + str(i) for i in range(100)]),
                                               np.array(['neg' + str(i) for i in range(100)])]),
                                    index=diff_access_sites)
anno_diff = annotation.loc[set(diff_access_sites) & set(annotation.index)]
anno_diff['relation'] = diff_access_sites_df.loc[anno_diff.index]
anno_diff = anno_diff.sort_values(by='relation')
# anno_diff.to_csv('./SCARP_78659_peaks_genes.csv')